This notebook is part of an exploratory analysis of machine learning used to decompose hyperspectral datasets of hybrid perovskite nanoscale materials.

Two machine learning models are used: Nonnegative Matrix Factorization and Variational Autoencoders

**Notebook Four: Using Artificial Data**

# Imports, Functions, and Classes

## Imports

In [ ]:
%run ML_HSI_Analysis_1_Preprocessing.ipynb

In [1]:
import nimfa
import sklearn
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn import metrics

import torch; torch.manual_seed(0)
from torchvision.utils import make_grid
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import pyroved as pv
import pyro.distributions as dist
import pyro
from pyroved.models.base import baseVAE
from pyroved.nets import fcDecoderNet, fcEncoderNet, sDecoderNet
from pyroved.utils import (
    generate_grid, get_sampler, set_deterministic_mode, to_onehot, transform_coordinates)

# VAE

In [83]:
def vae_plot(z_mean, xpix, ypix, scalebar_size):
    
    rows = 2
    columns = len(z_mean[0])
    
    cmap = mpl.colormaps['viridis']
    fig = plt.figure(figsize=(8,8), dpi=600)
    if columns == 3:
        gs = GridSpec(rows, columns, top=0.5)
    else:
        gs = GridSpec(rows, columns)
    #gs_top = GridSpec(rows, columns, top=0.99)
    #gs_base = GridSpec(rows, columns, wspace=0.3, hspace=0.03, top=0.8, bottom=0.3)
    #gs_top.tight_layout(fig)
    #gs_base.tight_layout(fig)
    fig.patch.set_facecolor('white')
    
    for i in range(rows):
        column_count = 0
        for j in range(columns):
            if i == 0:
                if columns == 2:
                    ax = fig.add_subplot(gs[0,column_count])
                    ax.scatter(z_mean[:,0], z_mean[:,1], c=z_mean[:,column_count], cmap=cmap,
                            ec='black', lw=0.2, alpha=0.6, s=3)
                    ax.set_xticks([0.25, 0.5, 0.75, 1])
                    ax.set_yticks([0.25, 0.5, 0.75, 1])
                    ax.set_xticklabels([0.25, 0.5, 0.75, 1], fontsize=5)
                    ax.set_yticklabels([0.25, 0.5, 0.75, 1], fontsize=5)
                if columns == 3:
                    ax = fig.add_subplot(gs[0,column_count], projection='3d')
                    ax.scatter(z_mean[:,0], z_mean[:,1], z_mean[:,2], 
                            c=z_mean[:,column_count], cmap=cmap,
                            ec='black', lw=0.05, alpha=0.5, s=3)
                    ax.set_xticks([0.25, 0.5, 0.75, 1])
                    ax.set_yticks([0.25, 0.5, 0.75, 1])
                    ax.set_zticks([0.25, 0.5, 0.75, 1])
                    ax.set_xticklabels([])
                    ax.set_yticklabels([])
                    ax.set_zticklabels([])
                    ax.set_xlabel("X", size=10, labelpad=-15)
                    ax.set_ylabel("Y", size=10, labelpad=-15)
                    ax.set_zlabel("Z", size=10, labelpad=-15)
                    #ax.set_zticklabels([0.25, 0.5, 0.75, 1], fontsize=5)
                    if j == 0:
                        ax.view_init(30, -90) 
                    elif j == 1:
                        ax.view_init(30, -30)
                    elif j == 2:
                        ax.view_init(30, 30)
                ax.tick_params(axis='both', which='minor', labelsize=20, direction='out', 
                               length=12, width=4)
                if column_count == 0:
                    ax.set_title('Latent Space', size=20)
            
            if i == 1:
                img = fig.add_subplot(gs[1,column_count])
                img.imshow(z_mean[:,column_count].reshape(ypix, xpix), cmap=cmap)
                img.set_xticks([])
                img.set_yticks([])
                
                if column_count == (columns-1):
                    scalebar = AnchoredSizeBar(img.transData, scalebar_size, " ", 
                            "lower right", pad=0.3, color='#F2F2F2', frameon=False,
                                               size_vertical=3, label_top=True)
                    img.add_artist(scalebar)

            column_count += 1
    
    #fig.savefig('VAE/3.2) Figure 1, Y-axis spectra (batch size 256).png')

In [84]:
# Spectra Plot (For determining latent dimensions)
def spectra_plot(img2, array, xpix, ypix, wav, title):
    
    n = xpix*ypix # total number of spectra
    cmap = mpl.colormaps['viridis'](np.linspace(0,1,n)) # create colormap
    fig = plt.figure(figsize=(8,8), dpi=600)
    fig.patch.set_facecolor('white')
    ax = fig.add_subplot()
    ax.set_prop_cycle('color', list(cmap)) # colormaps each spectra from first to last in array
    ax.set_title(title, size=20)
    plt.xlim(spect_range_b, spect_range_t)
    #ax.set_xticks([500, 750])
    #ax.set_xticklabels([])
    #plt.yscale('log')
    
    for i in array:
        ax.plot(wav, img2[:,i[0],i[1]], lw=0.3, alpha=0.5)
    
    #fig.savefig('VAE/4.3) Figure 1, 3D, Z-axis spectra (batch size 256).png')

In [90]:
f1_denoised_1d_transpose = f1_denoised_2d.transpose(1,0).reshape(f1_zpix*f1_ypix*f1_xpix)
f1_denoised_2d_norm = normalize(f1_denoised_1d_transpose).reshape(f1_ypix*f1_xpix, f1_zpix)

f1_denoised_2d_norm_t = torch.from_numpy(np.array(f1_denoised_2d_norm).astype('float64')).float()
f1_n_samples = f1_denoised_2d_norm_t.size()[0] # number of spectral points
f1_l_signal = f1_denoised_2d_norm_t.size()[1] # number of spectra

f1_train_data = f1_denoised_2d_norm_t.clone()
f1_train_loader = pv.utils.init_dataloader(f1_train_data.unsqueeze(1), batch_size=64)

f1_in_dim = (f1_l_signal,)
f1_kl_scale = torch.linspace(start=.001, end=.01, steps=50)

In [91]:
f1_vae = pv.models.iVAE(f1_in_dim, latent_dim=2, invariances=None, sampler_d="gaussian")
f1_vae_trainer = pv.trainers.SVItrainer(f1_vae)

for e in range(50):
    sc = f1_kl_scale[e] if e < len(f1_kl_scale) else f1_kl_scale[-1]
    f1_vae_trainer.step(f1_train_loader, scale_factor=sc)
    if e//10 == e/10:
        f1_vae_trainer.print_statistics()

Epoch: 1 Training loss: 250.9941
Epoch: 11 Training loss: 231.3356
Epoch: 21 Training loss: 231.2580
Epoch: 31 Training loss: 231.2618
Epoch: 41 Training loss: 231.2678


In [92]:
f1_vae_z_mean, f1_vae_z_sd = f1_vae.encode(f1_train_data)

In [93]:
f1_vae_z_mean = normalize(f1_vae_z_mean)
f1_vae_z_sd = normalize(f1_vae_z_sd)

# NMF and VAE on artificial data

## Reading Data

In [ ]:
from Functions import *

In [ ]:
f0 = np.load('Blazer Logo Hyperspectral Image.npy', allow_pickle=True)

In [ ]:
f0_xpix = f0.shape[2]
f0_ypix = f0.shape[1]
f0_zpix = f0.shape[0]

## Denoising

In [ ]:
f0_1d = np.reshape(f0, (f0_zpix*f0_ypix*f0_xpix))
# Subtract background
f0_sb_mean = np.mean(f0[len(f0[:,0,0])-101:len(f0[:,0,0])-1,470,300])
f0_1d = [(i - f0_sb_mean) for i in f0_1d]
f0_1d = [i.clip(min=0) for i in f0_1d]
# SB + Median
f0_1d = median_filter(
    f0_1d, size=3, footprint=None, output=None, mode='reflect', cval=0.0, origin=0)
# Set negative values equal to 0
for i in range(len(f0_1d)):
    if f0_1d[i] < 0:
        f0_1d[i] = 0
f0 = np.reshape(f0_1d, (f0_zpix, f0_ypix, f0_xpix))
f0_2d = np.reshape(f0_1d, (f0_zpix, f0_ypix*f0_xpix))

In [ ]:
def artificial_data_plot(hsi):
    # Plot figure
    fig = plt.figure(figsize=(18, 15))
    rows = 3 
    columns = 2
    cmap = 'viridis'

    # img1
    fig.add_subplot(rows, columns, 1)
    plt.imshow(hsi[230,:,], cmap=cmap)
    plt.title("Wavelength:230")
    plt.plot(300, 300, "X", markersize=10, c='red')
    plt.plot(450, 300, "X", markersize=10, c='blue')

    # plt1
    fig.add_subplot(rows, columns, 2)
    plt.plot(hsi[:,300,300], c='red')
    plt.plot(hsi[:,450,300], c='blue')
    plt.axvline(x=230, c='black', lw=2)

    # img2
    fig.add_subplot(rows, columns, 3)
    plt.imshow(hsi[130,:,:], cmap=cmap)
    plt.title("Wavelength:130")
    plt.plot(300, 475, "X", markersize=10, c='red')  
    plt.plot(247, 90,"X", markersize=10, c='blue')

    # plt2
    fig.add_subplot(rows, columns,4)
    plt.plot(hsi[:,300,475], c='red')
    plt.plot(hsi[:,247,90], c='blue')
    plt.axvline(x=130, c='black',lw=2)

    # img3
    fig.add_subplot(rows, columns, 5)
    plt.imshow(hsi[50,:,:], cmap=cmap)
    plt.title("Wavelength:50")
    plt.plot(100, 300, "X", markersize=10, c='red')  
    plt.plot(470,300,"X", markersize=10, c='blue')

    # plt3
    fig.add_subplot(rows,columns,6)
    plt.plot(hsi[:,100,300], c='red')
    plt.plot(hsi[:,470,300], c='blue')
    plt.axvline(x=50, c='black',lw=2)
    plt.show()

In [ ]:
artificial_data_plot(f0)

## NMF

In [ ]:
# 2 Components
f0_nmf_2comp = nmf_2comp
W_f0_nmf_2comp = f0_nmf_2comp.fit_transform(f0_2d)
H_f0_nmf_2comp = f0_nmf_2comp.components_
# 3 Components
f0_nmf_3comp = nmf_3comp
W_f0_nmf_3comp = f0_nmf_3comp.fit_transform(f0_2d)
H_f0_nmf_3comp = f0_nmf_3comp.components_
# 4 Components
f0_nmf_4comp = nmf_4comp
W_f0_nmf_4comp = f0_nmf_4comp.fit_transform(f0_2d)
H_f0_nmf_4comp = f0_nmf_4comp.components_
# 5 Components
f0_nmf_5comp = nmf_5comp
W_f0_nmf_5comp = f0_nmf_5comp.fit_transform(f0_2d)
H_f0_nmf_5comp = f0_nmf_5comp.components_
# 6 Components
f0_nmf_6comp = nmf_6comp
W_f0_nmf_6comp = f0_nmf_6comp.fit_transform(f0_2d)
H_f0_nmf_6comp = f0_nmf_6comp.components_

In [ ]:
# Spectral components
f0_nmf_spect_matrices = [W_f0_nmf_2comp, W_f0_nmf_3comp, W_f0_nmf_4comp, W_f0_nmf_5comp, 
                         W_f0_nmf_6comp]
# Image components
f0_nmf_img_matrices = [H_f0_nmf_2comp, H_f0_nmf_3comp, H_f0_nmf_4comp, H_f0_nmf_5comp, 
                       H_f0_nmf_6comp]

### Plot code

In [ ]:
def artificial_nmf_plot(suptitle, spect_matrices, img_matrices, ypix, xpix):

    rows = len(spect_matrices)
    columns = len(img_matrices)+3
    gs = GridSpec(rows, columns)
    gs.update(wspace=0.09,hspace=0)
    figaspect = plt.figaspect(float(ypix*3)/float(xpix*6))
    figsize = (20,32/(figaspect[0]/figaspect[1]))

    fig = plt.figure(figsize=figsize, dpi=800)
    fig.patch.set_facecolor('#00000000')
    fig.suptitle(suptitle, fontsize=30)

    row_count = 0
    for i in range(rows):

        fig_plt = fig.add_subplot(gs[row_count,:2])
        fig_plt.set_xticks([])
        fig_plt.set_yticks([])
        #fig_plt.set_title(str(row_count+2) + " spectral components")

        column_count = 0
        for j in range(row_count+2):

            fig_img = fig.add_subplot(gs[row_count,column_count+2])
            fig_img.imshow(img_matrices[row_count][column_count,:].reshape(ypix, xpix))
            #fig_img.set_title(f1_nmf_analysis_type[row_count] + str(column_count))
            fig_img.set_xticks([])
            fig_img.set_yticks([])
            fig_img.tick_params(color=color[j])
            for spine in fig_img.spines.values():
                spine.set_edgecolor(color[j])
                spine.set(lw=5)

            fig_plt.plot(spect_matrices[row_count][:,column_count], 
                         c=color[j], lw=3)
            fig_plt.set_yticks([])
            fig_plt.set_yticklabels([])
            #fig_plt.tick_params(axis='both', direction='out', length=12, width=4)
            if row_count == rows-1:
                fig_plt.set_xticks([])
                fig_plt.set_xticklabels([])

            column_count += 1

        row_count += 1
    
    #fig.savefig('Graphs/final/3.1) Blind NMF plotting 2-6 components (sklearn).svg')

### Plot

In [ ]:
artificial_nmf_plot('Blind NMF plotting 2-6 components', f0_nmf_spect_matrices, 
         f0_nmf_img_matrices, f0_ypix, f0_xpix)

### Sparseness

In [ ]:
snmf_n_comp = 3
snmf_iterations = 200
snmf_sparseness_levels = [1e-8, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0]

f0_nimfa_spect_snmf_matrices = []
#f0_nimfa_img_snmf_matrices = []

In [ ]:
for i in snmf_sparseness_levels:
    f0_nimfa_spect_snmf_matrices.append(snmf_model(f0_2d, i, snmf_n_comp, 
                                                   snmf_iterations, 'l'))

In [ ]:
snmf_plot("Sparseness in spectral components\nFigure 0", f0_nimfa_spect_snmf_matrices, 
          snmf_sparseness_levels, snmf_n_comp, f0_xpix, f0_ypix)